In [ ]:
import socket
import Utils as ut
import Statistics as st

rn = 0  # Sequence number
receiver = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
receiver.connect(st.ADDR)

def is_corrupted(data, crc):
    return ut.crc4itu(data) != crc

def send_ack():
    ack_frame = ut.makeFrame(rn, "11110000")
    receiver.send(ack_frame.encode())
    print(f"[ACK SENT] ACK {rn}")

def receive():
    global rn
    while True:
        print("[LISTENING] Receiver is listening...")
        frame = receiver.recv(20).decode()
        recv_n, _, data, crc = ut.receiveFrame(frame)
        
        if data == 'q':  # Check for termination
            print("[CLOSING] Closing receiver...")
            receiver.close()
            break

        print(f"[RECEIVED] Frame: {recv_n}, Data: {data}")

        if recv_n == rn:
            if not is_corrupted(data, crc):  # Check CRC
                print(f"[CRC SUCCESS] CRC {crc} matches.")
                send_ack()
                rn += 1
            else:
                print(f"[CRC FAILURE] Mismatch: Calculated {ut.crc4itu(data)}, Received {crc}")
        else:
            print("[OUT OF ORDER] Received wrong frame.")
        print("------------------------------------------------")

receive()
